# Create an Interactive Confusion Matrix in Weave

In this tutorial, we will set up an interactive confusion matrix Weave panel using Weave expressions and Python):
* pick an existing W&B Table of predictions
* load this data into Weave locally
* programmatically configure two Weave Panels: a Facet panel for the confusion matrix grid and an active selection panel to show the details of items in that cell of the confusion matrix

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/conf_mat_vert.png" width=70%>

# Step 0: Setup

Install dependencies and login to W&B to save & publish your work.

In [ ]:
# if not already installed
!pip install -qqq weave

In [ ]:
import wandb
wandb.login()
import weave
import weave.legacy.weave.panels

Use our public example or substitute your own W&B Tables by replacing the fields below. You can find the relevant information by navigating to a project in W&B, clicking on the "Artifacts" tab, and browsing using the sidebar until you find the artifact type and corresponding table file of interest (e.g. [this one used in the example](https://wandb.ai/stacey/mendeleev/artifacts/test_results/test_res_1fwmcd3q/v0/files/test_results.table.json)).

**Note:** the index '0' after `artifacts()` and `versions()` will take the first element in those lists.

In [ ]:
ENTITY = "stacey"
PROJECT = "mendeleev"
ARTIFACT_TYPE = "test_results"
TABLE_FILE = "test_results.table.json"

# Step 1: Compose Weave expression to load W&B Table

Navigate the W&B object tree programmatically with Weave
* start with the specified W&B project
* fetch all artifacts of the specified type and take the first artifact
* fetch all the versions of that artifact and take the first one
* load the specified W&B Table file associated with that artifact version
* convert that file into a Weave table (with `.table.rows()`)

The second cell below will display an interactive Weave Panel of the corresponding Table. You can interact with this panel in the notebook as you would with the underlying W&B Table in the cloud UI: sort, filter, group, etc. You can also expand the panel to open it in a Weave Board in order to iterate further/save and publish your work.

In [ ]:
predictions = (weave.legacy.weave.ops.project(ENTITY, PROJECT)
         .artifactType(ARTIFACT_TYPE)
         .artifacts()[0]
         .versions()[0]
         .file(TABLE_FILE)
         .table()
         .rows())

In [ ]:
predictions

# Step 2: Configure a Weave Panel Group in code

We write a function to create a group of panels in Weave:
* Group panel: this is a container to arrange a set of panels — in our case, in equal sizes horizontally
* Facet panel: this is the main 2D grid of the confusion matrix, showing the `truth` column (correct label) on the x-axis and the `guess` column (model's prediction) on the y-axis
* each cell of the Facet Panel is again a Group with two layers of panels: 1) a Number panel showing the total number of rows/model predictions in that cell of the confusion matrix, and 2) a background Color panel with the count from 1) rescaled on a blue hue, so the more saturated the blue, the more items in that cell
* a final Table panel to show the full rows/details of any selected cell from the main Facet panel.

In [ ]:
def confusion_matrix(table_data):
    conf_mat = weave.legacy.weave.panels.Group(
        equalSize=True,
        preferHorizontal=True,
        items={
            'confusion': weave.legacy.weave.panels.Facet(
                             table_data,
                             x=lambda row: row["truth"],
                             y=lambda row: row["guess"],
                             select=lambda row: weave.legacy.weave.panels.Group(
                                 layoutMode='layer',
                                 items={
                                     'count': row.count(),
                                     'color': weave.legacy.weave.panels.Color(row.count() / 50)
                                 }
                             )
                        ),
            'selected': lambda confusion: confusion.selected()
        }
    )
    return conf_mat

In [ ]:
confusion_matrix(predictions)

# Step 3: Filter for incorrect predictions

We can add a filter predicate to the Weave expression to select only the rows where the model's prediction doesn't match the correct label.  Note that the highest-count / most-blue squares are absent in this version.

Try opening this panel as a Weave Board by hovering over the right side and clicking "Open in new tab". You can explore more cells, add more panels, and share your work. Let us know if you find any interesting mistakes or patterns&mdash;for example, the mammal confused for an arachnid :)

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/conf_mat_full_board.png">

In [ ]:
mistakes_only = predictions.filter(lambda row: row["truth"] != row["guess"])
confusion_matrix(mistakes_only)

# Bonus: Make a confusion matrix from the UI only

To create a confusion matrix from the UI only:
1. Start with a Weave Table object saved with `weave.save()` and loaded into a panel with an expression of the form `get("wandb-artifact:///your_entity/your_project/obj:0123456789abcde/obj")`.
2. Click the pencil "Edit" icon on the panel to open the panel config menu.
3. Change the Panel type to "Facet"&mdash;this is the main confusion matrix layout.
4. Set the X and Y axes for the intended confusion matrix, e.g to `row["truth"]` and `row["guess"]` for this example.
5. In the lower "Properties" section, change the child panel type from "Expression" to "Group", then change the layout from "vertical" to "layer" and select "Add child"&mdash;each child is a cell of the confusion matrix which "groups" or "layers" the count of items in that cell and the corresponding background color of the cell.
6. The "Input" of one child can be the count: enter `row.count` as the Weave expression. When you hit "Run", "Panel type" should update to "Number" and you should see numbers render in the confusion matrix panel.
7. Click "Add Child" and optionally add the cell background. Change "Panel type" to "Color" and enter `row.count / N` as the "Input", where N is some normalizing constant to scale your values from 0 for white to ~1.5 for the highest-saturation blue.

Your settings menu should look something like this:

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/conf_mat_UI_settings.png" width=25%>